# Задание 4
Всё это время мы анализировали наши выборки целиком — и тех пользователей, на которых алгоритм повлиял, и тех, кого он не мог затронуть (меньше 30 просмотров). А что, если мы будем отбирать только нужных пользователей и скармливать t-тесту именно их? Да, выборка будет меньше, но мы избавимся от мусора — а значит, и чувствительность наверняка будет выше. В ответе укажите получившуюся мощность.

#### Выгрузим данные 

In [33]:
!pip install pandahouse==0.2.0

In [43]:
!pip install clickhouse-driver

In [5]:
!pip install tqdm

In [3]:
import pandas as pd
import pandahouse as ph
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import norm
from scipy.stats import ttest_ind
import numpy as np
from tqdm import tqdm



In [5]:
connection = {'host': 'https://clickhouse.lab.karpov.courses',
'database':'simulator_20250120',
'user':'student',
'password':'dpo_python_2020'}

In [7]:
q = """
select views, count() as users
from (select user_id,
             sum(action = 'view') as views
from {db}.feed_actions 
where toDate(time) BETWEEN '2024-12-27' AND  '2025-01-02'

   
group by user_id
)
group by views
order by views
"""

In [13]:
views_distribution = ph.read_clickhouse(q, connection=connection)
views_distribution['p'] = views_distribution['users']/views_distribution.users.sum()
views_distribution.sort_values(by = 'p', ascending = False)

,views,users,p
15,16,545,0.012977
14,15,537,0.012787
13,14,500,0.011906
34,35,485,0.011548
29,30,469,0.011167
...,...,...,...
280,287,1,0.000024
278,285,1,0.000024
276,280,1,0.000024
1,2,1,0.000024


In [15]:
views_distr = stats.rv_discrete(name='views_distr', 
                                values=(views_distribution['views'], 
                                        views_distribution['p']))

In [17]:
q_1 = """
select 
   floor(ctr, 2) as ctr, count() as users
from (select toDate(time) as dt, 
    user_id,
    sum(action = 'like')/sum(action = 'view') as ctr
from {db}.feed_actions
where dt BETWEEN '2024-12-27' AND  '2025-01-02'

group by dt, user_id
)
group by ctr
"""

In [19]:
ctr_distribution = ph.read_clickhouse(q_1, connection=connection)
ctr_distribution['p'] = ctr_distribution['users']/ctr_distribution.users.sum()
ctr_distribution.sort_values(by = 'p', ascending = False)

,ctr,users,p
17,0.20,4993,0.058658
33,0.16,4233,0.049729
50,0.25,4216,0.049529
6,0.18,4213,0.049494
72,0.21,3957,0.046487
...,...,...,...
11,0.81,2,0.000023
73,0.83,1,0.000012
51,1.00,1,0.000012
19,0.73,1,0.000012


In [21]:
views = views_distribution['views']

In [23]:
views_prob = views_distribution['users'] / views_distribution['users'].sum()

In [25]:
ctrs = ctr_distribution['ctr']

In [27]:
ctrs_prob = ctr_distribution['users'] / ctr_distribution['users'].sum()

In [29]:
rng = np.random.default_rng()

In [31]:
n_users = 30591

In [33]:
pvalues = []
for _ in tqdm(range(20000)):
    group_A_views = rng.choice(views_distribution['views'], size=n_users, replace=True, p=views_distribution['p']).astype(np.int64)
    group_B_views = rng.choice(views_distribution['views'], size=n_users, replace=True, p=views_distribution['p']).astype(np.int64)
    group_B_views += ((1 + rng.binomial(n=1, p=0.5, size=n_users)) * rng.binomial(n=1, p=0.9, size=n_users) * (group_B_views >= 30))
    group_A_ctrs = rng.choice(ctrs, size=n_users, replace=True, p=ctrs_prob)
    group_B_ctrs = rng.choice(ctrs, size=n_users, replace=True, p=ctrs_prob)
    mask_A = group_A_views >= 30
    mask_B = group_B_views >= 30
    group_A_likes = rng.binomial(n=group_A_views, p=group_A_ctrs)
    group_B_likes = rng.binomial(n=group_B_views, p=group_B_ctrs)
   

    _, p_value = ttest_ind(group_A_likes[mask_A], group_B_likes[mask_B], equal_var=False)
    pvalues.append(p_value)

# Вычисление доли значимых различий
alpha = 0.05
print(np.mean(np.array(pvalues) < alpha) * 100)


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [09:10<00:00, 36.31it/s]

64.495


In [ ]:
Ответ: 65.19